# Transfer Learning Model 'Xception Dropout' #

In [2]:

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.applications.xception import Xception


In [3]:
def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir, 'train')
     valid_path = os.path.join(base_dir, 'valid')

     # instantiate data generators
     datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

# the size of the images in the PCAM dataset
IMAGE_SIZE = 96
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
train_gen, val_gen = get_pcam_generators('C:/Users/s165635/Desktop/Project Imaging (8P361)/Dataset CHAMELYON16')
input = Input(input_shape)
# get the pretrained model, cut out the top layer
pretrained = Xception(input_shape=input_shape, include_top=False, weights='imagenet')

#for layer in pretrained.layers[-4]:
#    layer.trainable = False

# Freeze the layers except the last 4 layers
#for i, layer in enumerate(pretrained.layers):
    #layer.trainable = False
#    print(layer.name)

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [6]:
layernames = []
for i,layer in enumerate(pretrained.layers):
    layernames.append(layer.name)

for dropvalue in np.arange(0,1,0.05):
    dropvalue = np.around(dropvalue,2)
    pretrained = Xception(input_shape=input_shape, include_top=False, weights='imagenet')

    output = pretrained(input)
    output = GlobalAveragePooling2D()(output)
    output = Dropout(dropvalue)(output)
    output = Dense(1, activation='sigmoid')(output)

    # Check the trainable status of the individual layers
    #for layer in pretrained.layers:
    #    print(layer, layer.trainable)

    model = Model(input, output)

    # note the lower lr compared to the cnn example
    model.compile(SGD(lr=0.001, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])
        
    # save the model and weights
    string = 'DropOut'+str(dropvalue)
    print(string)
    model.summary()
    
    model_name = 'Xception'+string
    model_filepath = model_name + '.json'
    weights_filepath = model_name + '_weights.hdf5'

    model_json = model.to_json() # serialize model to JSON
    with open(model_filepath, 'w') as json_file:
        json_file.write(model_json)


    # define the model checkpoint and Tensorboard callbacks
    checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    tensorboard = TensorBoard(os.path.join('logs', model_name))
    callbacks_list = [checkpoint, tensorboard]


    # train the model, note that we define "mini-epochs"
    train_steps = train_gen.n//train_gen.batch_size//20
    val_steps = val_gen.n//val_gen.batch_size//20

    # since the model is trained for only 10 "mini-epochs", i.e. half of the data is
    # not used during training

    history = model.fit_generator(train_gen, steps_per_epoch=train_steps,
                        validation_data=val_gen,
                        validation_steps=val_steps,
                        epochs=1,
                        callbacks=callbacks_list)

    

DropOut0.0
0 input_4 False
1 block1_conv1 True
2 block1_conv1_bn True
3 block1_conv1_act True
4 block1_conv2 True
5 block1_conv2_bn True
6 block1_conv2_act True
7 block2_sepconv1 True
8 block2_sepconv1_bn True
9 block2_sepconv2_act True
10 block2_sepconv2 True
11 block2_sepconv2_bn True
12 conv2d_9 True
13 block2_pool True
14 batch_normalization_9 True
15 add_25 True
16 block3_sepconv1_act True
17 block3_sepconv1 True
18 block3_sepconv1_bn True
19 block3_sepconv2_act True
20 block3_sepconv2 True
21 block3_sepconv2_bn True
22 conv2d_10 True
23 block3_pool True
24 batch_normalization_10 True
25 add_26 True
26 block4_sepconv1_act True
27 block4_sepconv1 True
28 block4_sepconv1_bn True
29 block4_sepconv2_act True
30 block4_sepconv2 True
31 block4_sepconv2_bn True
32 conv2d_11 True
33 block4_pool True
34 batch_normalization_11 True
35 add_27 True
36 block5_sepconv1_act True
37 block5_sepconv1 True
38 block5_sepconv1_bn True
39 block5_sepconv2_act True
40 block5_sepconv2 True
41 block5_sepco

Epoch 1/1
225/225 [==============================] - 176s 783ms/step - loss: 0.5354 - acc: 0.7356 - val_loss: 0.3838 - val_acc: 0.8225

Epoch 00001: val_loss improved from inf to 0.38376, saving model to XceptionDropOut0.05_weights.hdf5
DropOut0.1
0 input_6 False
1 block1_conv1 True
2 block1_conv1_bn True
3 block1_conv1_act True
4 block1_conv2 True
5 block1_conv2_bn True
6 block1_conv2_act True
7 block2_sepconv1 True
8 block2_sepconv1_bn True
9 block2_sepconv2_act True
10 block2_sepconv2 True
11 block2_sepconv2_bn True
12 conv2d_17 True
13 block2_pool True
14 batch_normalization_17 True
15 add_49 True
16 block3_sepconv1_act True
17 block3_sepconv1 True
18 block3_sepconv1_bn True
19 block3_sepconv2_act True
20 block3_sepconv2 True
21 block3_sepconv2_bn True
22 conv2d_18 True
23 block3_pool True
24 batch_normalization_18 True
25 add_50 True
26 block4_sepconv1_act True
27 block4_sepconv1 True
28 block4_sepconv1_bn True
29 block4_sepconv2_act True
30 block4_sepconv2 True
31 block4_sepconv2

KeyboardInterrupt: 